In [1]:
%%time
import math
import torch
import numpy as np

from botorch.test_functions import SixHumpCamel
from botorch.fit import fit_gpytorch_mll
from botorch.models import SingleTaskGP
from botorch.utils.transforms import standardize, normalize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import (
    ExpectedImprovement,
    ProbabilityOfImprovement,
    qMaxValueEntropy,
)
from botorch.acquisition.max_value_entropy_search import qLowerBoundMaxValueEntropy
import matplotlib.pyplot as plt
from botorch.optim import optimize_acqf
from botorch.acquisition import qNoisyExpectedImprovement, PosteriorMean, qSimpleRegret
from time import time
import cProfile, io, pstats
from pstats import SortKey
import scipy
from scipy.optimize import minimize, Bounds

SMOKE_TEST = True

# torch.set_num_interop_threads(1)
# torch.set_num_threads(1)

torch.manual_seed(123456)
torch.set_default_dtype(torch.double)

bounds = torch.tensor(SixHumpCamel._bounds).T
bounds_norm = torch.tensor([[0.0, 0.0], [1.0, 1.0]])
train_X = bounds[0] + (bounds[1] - bounds[0]) * torch.rand(5, 2)
train_Y = SixHumpCamel(negate=True)(train_X).unsqueeze(-1)

train_X = normalize(train_X, bounds=bounds)
train_Y = standardize(train_Y + 0.05 * torch.randn_like(train_Y))

model = SingleTaskGP(train_X, train_Y)
mll = ExactMarginalLogLikelihood(model.likelihood, model)
fit_gpytorch_mll(mll, max_attempts=1)

# acqf = qNoisyExpectedImprovement(model, train_X, cache_root=False)
acqf = qSimpleRegret(model)

scipy.__version__

[KeOps] Warning : omp.h header is not in the path, disabling OpenMP. To fix this, you can set the environment
                  variable OMP_PATH to the location of the header before importing keopscore or pykeops,
                  e.g. using os.environ: import os; os.environ['OMP_PATH'] = '/path/to/omp/header'
[KeOps] Warning : Cuda libraries were not detected on the system or could not be loaded ; using cpu only mode
CPU times: user 2.79 s, sys: 823 ms, total: 3.61 s
Wall time: 4.85 s


'1.15.0'

In [2]:
# %%time
# print("\n\n\n\n\n\n optimize_acqf \n\n")

# # eval and maximise acq functions
# with cProfile.Profile() as pr:
#     optimize_acqf(
#         acq_function=acqf,
#         bounds=bounds_norm,
#         q=5,
#         num_restarts=5,
#         raw_samples=5,
#         sequential=False,
#     )

# sortby = SortKey.CUMULATIVE
# ps = pstats.Stats(pr).sort_stats(sortby)
# ps.print_stats(.1)

# scipy.__version__

In [3]:
%%time
print("\n\n\n\n\n\n calling minimize directly \n\n")


def f_and_grad(x):
    X_tensor = torch.from_numpy(x.reshape((5, 5, 2))).contiguous().requires_grad_(True)
    loss = acqf(X_tensor).sum()
    grad = torch.autograd.grad(loss, X_tensor)[0].reshape(-1).numpy(force=True)
    return loss.item(), grad

bounds = Bounds(
    lb=np.zeros(50),
    ub=np.ones(50),
    keep_feasible=True,
)
x0 = np.random.random(50)

with cProfile.Profile() as pr:
    res = minimize(
        fun=f_and_grad,
        x0=x0,
        method="L-BFGS-B",
        jac=True,
        bounds=bounds,
        options={"maxfun": 100}
    )

sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr).sort_stats(sortby)
ps.print_stats(.1)

scipy.__version__







 calling minimize directly 




/opt/anaconda3/envs/botorch/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(


         271755 function calls (258874 primitive calls) in 0.340 seconds

   Ordered by: cumulative time
   List reduced from 578 to 58 due to restriction <0.1>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.340    0.340 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_minimize.py:53(minimize)
        1    0.001    0.001    0.340    0.340 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_lbfgsb_py.py:290(_minimize_lbfgsb)
      204    0.002    0.000    0.335    0.002 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_optimize.py:71(_compute_if_needed)
      104    0.000    0.000    0.334    0.003 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_differentiable_functions.py:293(_update_fun)
      102    0.000    0.000    0.334    0.003 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_differentiable_functions.py:16(

'1.15.0'

In [4]:
%%time
print("\n\n\n\n\n\n 100 f_and_grad evaluations \n\n")

with cProfile.Profile() as pr:
    for _ in range(100):
        f_and_grad(x=np.random.random((50,)))

# sortby = SortKey.CUMULATIVE
# ps = pstats.Stats(pr).sort_stats(sortby)
# ps.print_stats(.1)

scipy.__version__







 100 f_and_grad evaluations 


CPU times: user 1.38 s, sys: 330 ms, total: 1.71 s
Wall time: 308 ms


'1.15.0'